<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузки" data-toc-modified-id="Загрузки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузки</a></span></li><li><span><a href="#Функции" data-toc-modified-id="Функции-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Функции</a></span><ul class="toc-item"><li><span><a href="#Функции-загрузки" data-toc-modified-id="Функции-загрузки-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Функции загрузки</a></span></li><li><span><a href="#Подгрузка-новых-данных-и-предсказание-на-новых-данных" data-toc-modified-id="Подгрузка-новых-данных-и-предсказание-на-новых-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Подгрузка новых данных и предсказание на новых данных</a></span></li><li><span><a href="#Создание-признаков" data-toc-modified-id="Создание-признаков-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Создание признаков</a></span></li><li><span><a href="#Функции-обучения" data-toc-modified-id="Функции-обучения-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Функции обучения</a></span></li></ul></li><li><span><a href="#Грузим-статистику" data-toc-modified-id="Грузим-статистику-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Грузим статистику</a></span></li><li><span><a href="#Приведение-данных-к-норме" data-toc-modified-id="Приведение-данных-к-норме-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Приведение данных к норме</a></span></li><li><span><a href="#Очистка-и-создание-признаков,-обучение" data-toc-modified-id="Очистка-и-создание-признаков,-обучение-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Очистка и создание признаков, обучение</a></span></li></ul></div>

# Упрощенный вариант

## Загрузки

In [1]:
'''Системные'''
import os
from datetime import datetime, timedelta
import time
# from tqdm import tqdm 
from pybit.unified_trading import HTTP
import joblib
from dotenv import load_dotenv, find_dotenv

'''База'''
import talib
# import yfinance as yf
import pandas as pd
import numpy as np
import itertools


'''Графики'''
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html
import plotly.graph_objects as go
import matplotlib.pyplot as plt

'''Обучение'''
import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import early_stopping
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
# Определяем устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

'''Подготовка'''
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler

'''Торговые агенты'''
from backtesting import Backtest, Strategy


# Настройка уровня логирования Optuna
import logging
optuna.logging.set_verbosity(optuna.logging.ERROR)

from config import (
    TP,
    SL,
    LEVERAGE,
    LIMIT,
    RS, 
    COMMISSION_BUY,
    COMMISSION_SELL,
    DEPOSIT,
    WAY
    )

load_dotenv(find_dotenv())
api_key = os.getenv("api_key_bybit")
api_secret = os.getenv("api_secret_bybit")

C:\YandexDisk\Code\.neural_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\YandexDisk\Code\.neural_env\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
# Выводим информацию об устройстве
if device.type == 'cuda':
#     print(f"Используется GPU: {torch.cuda.get_device_name(0)}")
    print(f"Количество доступных GPU: {torch.cuda.device_count()}")
    print(f"Текущий GPU: {torch.cuda.current_device()}")
    print(f"Объем памяти GPU: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"Количество CUDA ядер: {torch.cuda.get_device_properties(0).multi_processor_count}")
    print(f"Максимальная поддерживаемая версия CUDA: {torch.cuda.get_device_capability(0)}")
    print(f"Версия CUDA, установленная в системе: {torch.version.cuda}")
else:
    print("Используется CPU")
    print(f"Количество доступных CPU: {torch.get_num_threads()}")

Количество доступных GPU: 1
Текущий GPU: 0
Объем памяти GPU: 17.18 GB
Количество CUDA ядер: 34
Максимальная поддерживаемая версия CUDA: (8, 9)
Версия CUDA, установленная в системе: 11.8


In [3]:
all_data = []
all_data_crypt = []
# Определяем конечную дату
END_DATE = datetime.today()

# Определяем начальную дату
START_DATE = END_DATE - timedelta(days=0.3   * 365)

# Вычисляем период между датами
PERIOD = END_DATE - START_DATE

# Если нужно вывести даты в формате 'YYYY-MM-DD'
END_DATE_STR = END_DATE.strftime('%Y-%m-%d')
START_DATE_STR = START_DATE.strftime('%Y-%m-%d')

INTERVAL = 5
COIN = 'DOGEUSDT'

TEST_SIZE = 0.2
THRESHOLD = 0.5

COMMISSION = COMMISSION_BUY + COMMISSION_SELL

## Функции

### Функции загрузки

In [4]:
def fetch_coin_data(period, interval, coin):
    # Проверка существования файла
    if os.path.exists('df_work.csv'):
        with open('df_work.csv', 'r') as file:
            df = pd.read_csv(file, index_col=0)
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce') 
    else:
        limit = LIMIT
        current_datetime = datetime.now()
        end_date = current_datetime
        start_date = end_date - period

        # Проверяем, нужно ли разбивать запрос на части
        results = []
        if period.total_seconds() / (interval * 60) > (limit - 100):
            print('Запрос частями, т.к. больше лимита')
            total_parts = int(period.total_seconds() / (interval * 60 * (limit - 100))) + 1
            for i in range(total_parts):
                part_start_date = start_date + timedelta(minutes=i * (limit - 100) * interval)
                part_end_date = part_start_date + timedelta(minutes=(limit - 100) * interval)
                if part_end_date > end_date:
                    part_end_date = end_date

                session = HTTP(
                    testnet=False,
                    api_key=api_key,
                    api_secret=api_secret,
                    )
                response = session.get_kline(
                    category="linear",
                    symbol=coin,
                    interval=str(interval),
                    start=int(part_start_date.timestamp() * 1000),
                    end=int(part_end_date.timestamp() * 1000),
                    limit=limit - 100,
                )
                if response.get('result', {}).get('list'):
                    results.extend(response['result']['list'])
        else:
            session = HTTP(
                    testnet=False,
                    api_key=api_key,
                    api_secret=api_secret,
                    )
            response = session.get_kline(
                category="linear",
                symbol=coin,
                interval=str(interval),
                start=int(start_date.timestamp() * 1000),
                end=int(end_date.timestamp() * 1000),
                limit=limit,
            )
            if response.get('result', {}).get('list'):
                results = response['result']['list']

        # Проверка и создание DataFrame
        if not results:
            raise ValueError("Не удалось получить данные из API.")

        dates = [datetime.fromtimestamp(int(item[0]) / 1000) for item in results]
        prices_open = [float(item[1]) for item in results]
        prices_max = [float(item[2]) for item in results]
        prices_min = [float(item[3]) for item in results]
        prices_close = [float(item[4]) for item in results]
        volume = [float(item[5]) for item in results]
        df = pd.DataFrame({
            'date': dates,
            'open': prices_open,
            'high': prices_max,
            'low': prices_min,
            'close': prices_close,
            'volume': volume,
        }).set_index('date')
        df.to_csv('df_work.csv')
    return df

### Подгрузка новых данных и предсказание на новых данных

In [5]:
def update_df_coin(df_existing, interval, coin):
    limit = LIMIT
    session = HTTP(
                testnet=False,
                api_key=api_key,
                api_secret=api_secret,
                )
    current_datetime = datetime.now()
    
    # Определяем последний доступный временной момент в локальных данных
    last_timestamp = df_existing.index.max() if not df_existing.empty else None
    start_date = (last_timestamp + timedelta(minutes=interval)) if last_timestamp else (current_datetime - timedelta(days=30))  # 30 дней по умолчанию
    
    # Формируем запрос к API только для недостающих данных
    response = session.get_kline(
        category="linear",
        symbol=coin,
        interval=str(interval),
        start=int(start_date.timestamp() * 1000),
        end=int(current_datetime.timestamp() * 1000),
        limit=limit
    )
    results = response.get('result', {}).get('list', [])
    
    # Проверка и создание DataFrame для новых данных
    if not results:
#         print("Новых данных нет.")
        return df_existing

    dates = [datetime.fromtimestamp(int(item[0]) / 1000) for item in results]
    prices_open = [float(item[1]) for item in results]
    prices_max = [float(item[2]) for item in results]
    prices_min = [float(item[3]) for item in results]
    prices_close = [float(item[4]) for item in results]
    volume = [float(item[5]) for item in results]
    df_new = pd.DataFrame({
        'Date': dates,
        'Open': prices_open,
        'High': prices_max,
        'Low': prices_min,
        'Vlose': prices_close,
        'Volume': volume,
    }).set_index('Date')

    # Объединяем старые и новые данные
    df_updated = pd.concat([df_existing, df_new]).drop_duplicates()
    return df_updated


### Создание признаков

In [6]:
# Функция для добавления признаков
def add_features(df, macd_fast=40, macd_slow=91, macd_signal=145, sma_fast=80, sma_slow=90, rsi_period=5):
    df = df.copy()  # Создаем копию, чтобы не изменять исходный df
    df.columns = df.columns.str.capitalize()
    df['Close'] = df['Close'].astype(float)  # Принудительное преобразование в float
    
    # MACD, SMA, RSI
    df['RSI'] = talib.RSI(df['Close'], timeperiod=rsi_period)
    df['MACD'], df['MACD_signal'], _ = talib.MACD(
        df['Close'], 
        fastperiod=macd_fast, 
        slowperiod=macd_slow, 
        signalperiod=macd_signal)
    df['SMA_Fast'] = talib.SMA(df['Close'], timeperiod=sma_fast)
    df['SMA_Slow'] = talib.SMA(df['Close'], timeperiod=sma_slow)

    # Сигналы
    df['Signal'] = np.where(
            (df['MACD'] > df['MACD_signal']) & 
            (df['SMA_Fast'] > df['SMA_Slow']) & 
            (df['RSI'] < 30) , 1,  # Покупка  
        np.where(
            (df['MACD'] < df['MACD_signal']) & 
            (df['RSI'] > 70) & 
            (df['SMA_Fast'] < df['SMA_Slow']), -1, 0)  #  Продажа / Ожидание   
    )

    # Дополнительные RSI
    df['RSI_1'] = talib.RSI(df['Close'], timeperiod=5)
    df['RSI_2'] = talib.RSI(df['Close'], timeperiod=15)
    df['RSI_3'] = talib.RSI(df['Close'], timeperiod=50)

    # MACD с другими параметрами
    df['MACD'], df['MACD_signal'], df['MACD_slow'] = talib.MACD(df['Close'], fastperiod=15, slowperiod=60, signalperiod=3)

    # SMA с разными периодами
    df['SMA_1'] = talib.SMA(df['Close'], timeperiod=7)
    df['SMA_2'] = talib.SMA(df['Close'], timeperiod=15)
    df['SMA_3'] = talib.SMA(df['Close'], timeperiod=30)
    df['SMA_4'] = talib.SMA(df['Close'], timeperiod=100)

    # Дополнительные признаки
    df['d_min_max'] = df['High'] - df['Low']
    df['d_open_max'] = df['Open'] - df['High']
    df['d_open_min'] = df['Open'] - df['Low']

    # Уровни поддержки и сопротивления
    periods = [14, 20, 50, 100, 200]
    df = calculate_levels(df, periods=periods)

    # Дата и временные признаки
    try:
        df["Date"] = pd.to_datetime(df["Date"])
        df.set_index('Date', inplace=True)
    except:
        pass

    df['weekday_number'] = df.index.weekday
    df['week_number'] = df.index.isocalendar().week

    # Динамика 1 час
    df['динамика 1T'] = df['Close'] - df['Open']
    df['динамика 1T'] = df['динамика 1T'].shift(1)

    # Импульс
    df['Импульс'] = 0
    limit_dynamics = np.percentile(df['динамика 1T'].dropna(), 30)
    df.loc[df['динамика 1T'] < limit_dynamics, 'Импульс'] = 1

    # Дополнительные технические индикаторы
    df['adx'] = talib.ADX(df['High'], df['Low'], df['Close'], timeperiod=9)
    df['atr'] = talib.ATR(df['High'], df['Low'], df['Close'], timeperiod=9)
    df['atr_norm'] = df['atr'] / df['Close']

    df['upper_band'], df['middle_band'], df['lower_band'] = talib.BBANDS(df['Close'], timeperiod=7)
    df['bb_width'] = (df['upper_band'] - df['lower_band']) / df['middle_band']

    df['ema20'] = talib.EMA(df['Close'], timeperiod=20)
    df['ema50'] = talib.EMA(df['Close'], timeperiod=50)
    df['ema200'] = talib.EMA(df['Close'], timeperiod=200)

    df['macd_2'], df['macd_signal_2'], df['macd_hist_2'] = talib.MACD(df['Close'], fastperiod=15, slowperiod=20, signalperiod=9)

    df['sma_fast_2'] = talib.SMA(df['Close'], timeperiod=9)
    df['sma_slow_2'] = talib.SMA(df['Close'], timeperiod=30)
    df['tema'] = talib.TEMA(df['Close'], timeperiod=12)

    # Лаги
    df = create_lagged_features(df, 'tema', [2, 30])
    df = create_lagged_features(df, 'RSI_1', [2, 30])
    df = create_lagged_features(df, 'динамика 1T', [2, 30])
    df = create_lagged_features(df, 'Импульс', [2, 30])

    return df 

def calculate_levels(data, periods=[14]):
    for period in periods:
        high_col = f'high_max_{period}'
        low_col = f'low_min_{period}'

        data[high_col] = data['High'].rolling(window=period).max()
        data[low_col] = data['Low'].rolling(window=period).min()

    return data

def create_lagged_features(data, column, lag_range):
    """
    Создает лагированные признаки для указанного столбца.

    :param data: DataFrame с исходными данными.
    :param column: Название столбца, для которого нужно создать лаги.
    :param lag_range: Список [начало, конец] или кортеж (начало, конец) диапазона лагов.
    :return: DataFrame с добавленными лагированными колонками.
    """
    start, end = lag_range
    for i in range(start, end + 1):
        lag_col = f"{column}_lag_{i}h"
        data[lag_col] = data[column].shift(freq='h', periods=i)
    
    return data

### Функции обучения

In [7]:
# # Определяем нейросеть
# class SignalClassifier(nn.Module):
#     def __init__(self, input_dim, num_classes):
#         super(SignalClassifier, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 256)
#         self.fc2 = nn.Linear(256, 32)
#         self.fc3 = nn.Linear(32, num_classes)
#         self.relu = nn.ReLU()
#         self.softmax = nn.Softmax(dim=1)

#     def forward(self, x):
#         x = self.relu(self.fc1(x))
#         x = self.relu(self.fc2(x))
#         x = self.softmax(self.fc3(x))
#         return x
class SignalClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, dropout_rate=0.3):  # Добавили dropout_rate
        super(SignalClassifier, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256) # Добавили Batch Normalization после FC1
        self.dropout1 = nn.Dropout(dropout_rate) # Добавили слой Dropout после FC1
        self.fc2 = nn.Linear(256, 512) # Увеличили размер скрытого слоя
        self.bn2 = nn.BatchNorm1d(512) # Добавили Batch Normalization после FC2
        self.dropout2 = nn.Dropout(dropout_rate) # Добавили слой Dropout после FC2
        self.fc3 = nn.Linear(512, 8) # Добавили еще один скрытый слой
        self.bn3 = nn.BatchNorm1d(8) # Добавили Batch Normalization после FC3
        self.dropout3 = nn.Dropout(dropout_rate) # Добавили слой Dropout после FC3
        self.fc4 = nn.Linear(8, num_classes) # Адаптировали финальный слой
        self.relu = nn.ReLU()
        # Удалили softmax из модели. CrossEntropyLoss обрабатывает его
        # self.softmax = nn.Softmax(dim=1)


    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x))) # Добавили BN после первого Linear слоя
        x = self.dropout1(x) # Добавили dropout после первого Linear слоя
        x = self.relu(self.bn2(self.fc2(x))) # Добавили BN после второго Linear слоя
        x = self.dropout2(x) # Добавили dropout после второго Linear слоя
        x = self.relu(self.bn3(self.fc3(x))) # Добавили BN после третьего Linear слоя
        x = self.dropout3(x) # Добавили dropout после третьего Linear слоя
        x = self.fc4(x) # Удалили слой softmax
        #x = self.softmax(x) # Удалили слой softmax
        return x

# Функция обучения модели
# def train_pytorch_model(X_train, y_train, num_classes=3, model_filename="model.pth", epochs=50, batch_size=128):
#     X_train = torch.tensor(X_train.astype(np.float32).values, dtype=torch.float32).to(device)
#     y_train = torch.tensor(y_train.astype(np.int64).values, dtype=torch.long).to(device)
#     dataset = TensorDataset(X_train, y_train)
#     dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
#     model = SignalClassifier(X_train.shape[1], num_classes).to(device)
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=0.0001)
    
#     for epoch in range(epochs):
#         for batch_X, batch_y in dataloader:
#             batch_X, batch_y = batch_X.to(device), batch_y.to(device)
#             optimizer.zero_grad()
#             outputs = model(batch_X)
#             loss = criterion(outputs, batch_y)
#             loss.backward()
#             optimizer.step()
#         print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")
    
#     torch.save(model.state_dict(), model_filename)
#     print(f"Модель сохранена в {model_filename}")
#     return model

# def train_pytorch_model(X_train, y_train, num_classes=3, model_filename="model.pth", epochs=50, batch_size=128, learning_rate=0.0001, weight_decay=1e-5, clip_grad_norm=1.0):
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#     # 1. Преобразование данных в тензоры и перенос на устройство
#     X_train_tensor = torch.tensor(X_train.astype(np.float32).values, dtype=torch.float32).to(device)  # .values здесь!
#     y_train_tensor = torch.tensor(y_train.astype(np.int64).values, dtype=torch.long).to(device)   # .values здесь!

#     # 2. Проверка данных на NaN и Inf
#     if torch.isnan(X_train_tensor).any() or torch.isinf(X_train_tensor).any():
#         raise ValueError("X_train содержит NaN или Inf!")
#     if torch.isnan(y_train_tensor).any() or torch.isinf(y_train_tensor).any():
#         raise ValueError("y_train содержит NaN или Inf!")


#     # 3. Создание датасета и даталоадера
#     dataset = TensorDataset(X_train_tensor, y_train_tensor)
#     dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

#     # 4. Создание модели и перенос на устройство
#     model = SignalClassifier(X_train_tensor.shape[1], num_classes).to(device)

#     # 5. Инициализация весов
#     def init_weights(m):
#         if isinstance(m, nn.Linear):
#             torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

#     model.apply(init_weights)

#     # 6. Определение функции потерь и оптимизатора
#     criterion = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

#     # 7. Обучение модели
#     model.train()
#     for epoch in range(epochs):
#         total_loss = 0
#         for batch_X, batch_y in dataloader:
#             optimizer.zero_grad()
#             outputs = model(batch_X)
#             loss = criterion(outputs, batch_y)
#             loss.backward()
#             torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad_norm)
#             optimizer.step()

#             total_loss += loss.item()

#             if torch.isnan(loss).any():
#                 print("Обнаружен NaN loss! Остановка обучения.")
#                 return model

#         avg_loss = total_loss / len(dataloader)
#         print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

#     # 8. Сохранение модели
#     model.eval()
#     torch.save(model.state_dict(), model_filename)
#     print(f"Модель сохранена в {model_filename}")

#     return model


def train_pytorch_model(X_train, y_train, num_classes=3, model_filename="model.pth", epochs=50, batch_size=128, learning_rate=0.0001,
                        weight_decay=1e-5, clip_grad_norm=1.0):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 1. Преобразование данных в тензоры и перенос на устройство
    X_train_np = X_train.astype(np.float32).values  # Преобразуем DataFrame в numpy array
    y_train_np = y_train.astype(np.int64).values    # Преобразуем DataFrame в numpy array

    # **ПРОВЕРКА СРАЗУ ПЕРЕД СОЗДАНИЕМ ТЕНЗОРА**
    print("Непосредственно перед созданием тензора:")
    print("NaN в X_train_np:", np.isnan(X_train_np).sum())
    print("Inf в X_train_np:", np.isinf(X_train_np).sum())

    X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32).to(device)
    y_train_tensor = torch.tensor(y_train_np, dtype=torch.long).to(device)

    # 2. Проверка данных на NaN и Inf
    if torch.isnan(X_train_tensor).any() or torch.isinf(X_train_tensor).any():
        raise ValueError("X_train содержит NaN или Inf!")
    if torch.isnan(y_train_tensor).any() or torch.isinf(y_train_tensor).any():
        raise ValueError("y_train содержит NaN или Inf!")

    # 3. Создание датасета и даталоадера
    dataset = TensorDataset(X_train_tensor, y_train_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # 4. Создание модели и перенос на устройство
    model = SignalClassifier(X_train_tensor.shape[1], num_classes).to(device)

    # 5. Инициализация весов
    def init_weights(m):
        if isinstance(m, nn.Linear):
            torch.nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

    model.apply(init_weights)

    # 6. Определение функции потерь и оптимизатора
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    # 7. Обучение модели
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip_grad_norm)
            optimizer.step()

            total_loss += loss.item()

            if torch.isnan(loss).any():
                print("Обнаружен NaN loss! Остановка обучения.")
                return model

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {avg_loss:.4f}")

    # 8. Сохранение модели
    model.eval()
    torch.save(model.state_dict(), model_filename)
    print(f"Модель сохранена в {model_filename}")

    return model


# Функция предсказания
def predict_pytorch_model(X_test, model_filename="model.pth", num_classes=3):
    model = SignalClassifier(X_test.shape[1], num_classes).to(device)
    model.load_state_dict(torch.load(model_filename, map_location=device))
    model.eval()
    
    X_test = torch.tensor(X_test.astype(np.float32).values, dtype=torch.float32).to(device)
    with torch.no_grad():
        predictions = model(X_test)
    return np.argmax(predictions.cpu().numpy(), axis=1)

# # Подготовка данных
# def process_data(df_coin):
#     df_coin.columns = df_coin.columns.str.capitalize()
#     df_coin['Target'] = np.where(df_coin['Close'].shift(-1) > df_coin['Close'], 1, 0)
#     train_size = int(len(df_coin) * 0.9)
#     return df_coin[:train_size], df_coin[train_size:]


In [8]:
class MyStrategy(Strategy):
    model = None       # Параметр для модели ML
    features = None    # Параметр для списка признаков
    stop_loss = SL   # Уровень Stop Loss (_%)
    take_profit = TP  # Уровень Take Profit (_%)
    def init(self):
        self.signal = self.I(lambda: self.data.Signal)
        self.previous_signal = 0

    def next(self):
        """
        Основной цикл стратегии:
        - Используем сигналы для открытия/закрытия позиций
        """
        signal = self.data.df['Signal'].iloc[-1]  # Последний сигнал
        current_price = self.data.Close[-1]      # Текущая цена закрытия
        
        if signal == 1:  # Покупка
            self.buy(
                sl=current_price * (1 - self.stop_loss),  # Уровень Stop Loss
                tp=current_price * (1 + self.take_profit)  # Уровень Take Profit
            )
        elif signal == -1:  # Продажа
            self.sell(
                sl=current_price * (1 + self.stop_loss),  # Уровень Stop Loss
                tp=current_price * (1 - self.take_profit)  # Уровень Take Profit
            )
                

## Грузим статистику

In [9]:

data_temp =fetch_coin_data(PERIOD, INTERVAL, COIN)
if isinstance(data_temp.columns, pd.MultiIndex):
    data_temp.columns = data_temp.columns.droplevel([1])
data_temp = data_temp.reset_index(drop=False)
data_temp.index.name = 'Price'
#             data_temp.rename_axis(None, inplace=True)
data_temp.columns.name = None
data_temp = data_temp.reset_index(drop=True)
# df.columns = df.columns.droplevel(1)
data_temp.columns = data_temp.columns.str.lower()
data_temp.head(3)

Запрос частями, т.к. больше лимита


,date,open,high,low,close,volume
0,2024-11-07 05:15:00,0.19815,0.19859,0.19736,0.19818,24803112.0
1,2024-11-07 05:10:00,0.19851,0.19978,0.19785,0.19815,33520180.0
2,2024-11-07 05:05:00,0.19771,0.19892,0.19738,0.19851,15475290.0


## Приведение данных к норме

In [10]:
data = data_temp.copy() 
df = data.copy()
len(df)

31536

In [11]:
if not df.index.is_monotonic_increasing:
    print("Индекс не отсортирован. Сортируем...")
    df = df.sort_index()

In [12]:
df = df.fillna(method='ffill')
df = pd.DataFrame(df.replace(to_replace=0, method='ffill'))

C:\Users\fd\AppData\Local\Temp\ipykernel_21444\4089252463.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill')
C:\Users\fd\AppData\Local\Temp\ipykernel_21444\4089252463.py:2: FutureWarning: The 'method' keyword in DataFrame.replace is deprecated and will be removed in a future version.
  df = pd.DataFrame(df.replace(to_replace=0, method='ffill'))


In [13]:
len(df.loc[(df['close'] > df['close'].rolling(window=3).mean() * 1.5) | (df['close'] < df['close'].rolling(window=3).mean() * 0.6)])

0

## Очистка и создание признаков, обучение

In [14]:
df_coin = df.copy()
df_coin.head(2)

,date,open,high,low,close,volume
0,2024-11-07 05:15:00,0.19815,0.19859,0.19736,0.19818,24803112.0
1,2024-11-07 05:10:00,0.19851,0.19978,0.19785,0.19815,33520180.0


In [15]:
if not df.index.is_monotonic_increasing:
    print("Индекс не отсортирован. Сортируем...")
    df = df.sort_index()

In [16]:
df_coin.columns = df_coin.columns.str.capitalize()


# Добавляем целевую переменную
df_coin['Target'] = np.where(df_coin['Close'].shift(-1) > df_coin['Close'], 1, 0)


# Финальный бэктест с оптимальными параметрами
train_size = int(len(df_coin) * 0.8)
X_train, X_test = df_coin[:train_size], df_coin[train_size:]

In [17]:
df_coin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31536 entries, 0 to 31535
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    31536 non-null  datetime64[ns]
 1   Open    31536 non-null  float64       
 2   High    31536 non-null  float64       
 3   Low     31536 non-null  float64       
 4   Close   31536 non-null  float64       
 5   Volume  31536 non-null  float64       
 6   Target  31536 non-null  int32         
dtypes: datetime64[ns](1), float64(5), int32(1)
memory usage: 1.6 MB


In [18]:
# Применение функций
# best_params = optimize_hyperparameters(X_train)
df_final_tr = add_features(X_train).fillna(0)
df_final_tr['Signal'] = df_final_tr['Signal'] + 1

C:\Users\fd\AppData\Local\Temp\ipykernel_21444\1501981350.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[lag_col] = data[column].shift(freq='h', periods=i)
C:\Users\fd\AppData\Local\Temp\ipykernel_21444\1501981350.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[lag_col] = data[column].shift(freq='h', periods=i)
C:\Users\fd\AppData\Local\Temp\ipykernel_21444\1501981350.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

C:\Users\fd\AppData\Local\Temp\ipykernel_21444\1501981350.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[lag_col] = data[column].shift(freq='h', periods=i)
C:\Users\fd\AppData\Local\Temp\ipykernel_21444\1501981350.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[lag_col] = data[column].shift(freq='h', periods=i)
C:\Users\fd\AppData\Local\Temp\ipykernel_21444\1501981350.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

In [19]:
df_final_tr['Signal'].value_counts()

Signal
1    22828
2     1228
0     1172
Name: count, dtype: int64

In [20]:
# Создаем объект StandardScaler
scaler = StandardScaler()
X_train = X_train.drop(columns='Date')
# Обучаем scaler на тренировочных данных и трансформируем их
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)


model = train_pytorch_model(df_final_tr.drop(columns=['Signal']), df_final_tr['Signal'])

df_final = add_features(X_test)
X_test = X_test.drop(columns='Date')
# Применяем scaler к тестовым данным
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

predictions = predict_pytorch_model(df_final.drop(columns=['Signal']))

# df_final['Signal'] = predictions - 1
predicted_signal = predictions #- 1
df_final['Signal_pred'] = predicted_signal
# df_final['Signal'] = np.where(df_final['Signal'] != df_final['Signal_pred'], 0, df_final['Signal'])

df_final = df_final.fillna(0)
bt = Backtest(df_final, MyStrategy, cash=1000, commission=0.001, exclusive_orders=True)
stats = bt.run()
print(stats[:27])


Непосредственно перед созданием тензора:
NaN в X_train_np: 0
Inf в X_train_np: 0
Epoch 1/50, Loss: 1.3140
Epoch 2/50, Loss: 1.0775
Epoch 3/50, Loss: 0.9393
Epoch 4/50, Loss: 0.8390
Epoch 5/50, Loss: 0.7627
Epoch 6/50, Loss: 0.7031
Epoch 7/50, Loss: 0.6579
Epoch 8/50, Loss: 0.6240
Epoch 9/50, Loss: 0.5944
Epoch 10/50, Loss: 0.5689
Epoch 11/50, Loss: 0.5546
Epoch 12/50, Loss: 0.5367
Epoch 13/50, Loss: 0.5243
Epoch 14/50, Loss: 0.5112
Epoch 15/50, Loss: 0.5069
Epoch 16/50, Loss: 0.4950
Epoch 17/50, Loss: 0.4896
Epoch 18/50, Loss: 0.4881
Epoch 19/50, Loss: 0.4854
Epoch 20/50, Loss: 0.4741
Epoch 21/50, Loss: 0.4612
Epoch 22/50, Loss: 0.4607
Epoch 23/50, Loss: 0.4544
Epoch 24/50, Loss: 0.4483
Epoch 25/50, Loss: 0.4436
Epoch 26/50, Loss: 0.4398
Epoch 27/50, Loss: 0.4382
Epoch 28/50, Loss: 0.4388
Epoch 29/50, Loss: 0.4289
Epoch 30/50, Loss: 0.4307
Epoch 31/50, Loss: 0.4333
Epoch 32/50, Loss: 0.4295
Epoch 33/50, Loss: 0.4264
Epoch 34/50, Loss: 0.4256
Epoch 35/50, Loss: 0.4226
Epoch 36/50, Loss:

C:\Users\fd\AppData\Local\Temp\ipykernel_21444\1501981350.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[lag_col] = data[column].shift(freq='h', periods=i)
C:\Users\fd\AppData\Local\Temp\ipykernel_21444\1501981350.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data[lag_col] = data[column].shift(freq='h', periods=i)
C:\Users\fd\AppData\Local\Temp\ipykernel_21444\1501981350.py:118: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor

C:\Users\fd\AppData\Local\Temp\ipykernel_21444\2931575996.py:23: UserWarning: Data index is not sorted in ascending order. Sorting.
  bt = Backtest(df_final, MyStrategy, cash=1000, commission=0.001, exclusive_orders=True)


Start                     2025-01-30 14:20:00
End                       2025-02-21 14:15:00
Duration                     21 days 23:55:00
Exposure Time [%]                   56.166772
Equity Final [$]                  1213.541605
Equity Peak [$]                   1348.465364
Return [%]                           21.35416
Buy & Hold Return [%]              -23.410038
Return (Ann.) [%]                 2258.687465
Volatility (Ann.) [%]              3085.96566
Sharpe Ratio                         0.731922
Sortino Ratio                       64.234332
Calmar Ratio                       225.739471
Max. Drawdown [%]                  -10.005727
Avg. Drawdown [%]                   -1.043856
Max. Drawdown Duration        9 days 21:40:00
Avg. Drawdown Duration        0 days 12:32:00
# Trades                                  591
Win Rate [%]                        44.162437
Best Trade [%]                       1.901902
Worst Trade [%]                     -0.600601
Avg. Trade [%]                    

In [21]:
# print(list(df_final.columns))
df_final.Signal.unique()

array([ 0, -1,  1])

In [22]:
bt.plot(
    plot_equity=True, 
    plot_drawdown=True, 
    relative_equity=False,
    resample=False  # or resample='5Min',  or resample='H' etc.
    )
    

C:\YandexDisk\Code\.neural_env\lib\site-packages\backtesting\_plotting.py:455: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df2 = (df.assign(_width=1).set_index('datetime')


GridPlot(id='p1644', ...)